In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop, Adam, Adamax
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Load the data into Pandas dataframe

In [6]:
df = pd.read_csv('/kaggle/input/sms-spam-collection-dataset/spam.csv',delimiter=',',encoding='latin-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


## Drop the columns that are not required for the neural network.

In [7]:
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],axis=1,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


* Create input and output vectors.
* Process the labels.

In [10]:
X = df.v2
Y = df.v1
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

## Split into training and test data.

In [12]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

## Process the data
* Tokenize the data and convert the text to sequences.
* Add padding to ensure that all the sequences have the same shape.
* There are many ways of taking the max_len and here an arbitrary length of 150 is chosen.

In [15]:
from keras.utils import pad_sequences
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = pad_sequences(sequences,maxlen=max_len)

## RNN
* Define the RNN structure.

In [16]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,75,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(512,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.6)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

# Call the function and compile the model.

In [17]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=Adamax(),metrics=['accuracy'])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 150)]             0         
                                                                 
 embedding (Embedding)       (None, 150, 75)           75000     
                                                                 
 lstm (LSTM)                 (None, 64)                35840     
                                                                 
 FC1 (Dense)                 (None, 512)               33280     
                                                                 
 activation (Activation)     (None, 512)               0         
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 513   

# Fit on the training data.

In [20]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=2,
          validation_split=0.5,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0005)])

Epoch 1/2
19/19 [==============================] - 5s 291ms/step - loss: 0.1195 - accuracy: 0.9742 - val_loss: 0.1066 - val_accuracy: 0.9700
Epoch 2/2
19/19 [==============================] - 6s 321ms/step - loss: 0.0733 - accuracy: 0.9818 - val_loss: 0.0743 - val_accuracy: 0.9780


## Process the test set data.

In [22]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = pad_sequences(test_sequences,maxlen=max_len)

## Evaluate the model on the test set.

In [23]:
accr = model.evaluate(test_sequences_matrix,Y_test)

27/27 [==============================] - 1s 26ms/step - loss: 0.0788 - accuracy: 0.9785


In [24]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 0.079
  Accuracy: 0.978
